In [2]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import matplotlib.pyplot as plt

# Author: Joseph Ballai



# Assumptions Made:  I decided that anytime the algorithm would buy a stock, I would allocate 100% of my portfolio (equal to one share of MSFT).
#                    I only took long positions so I did not factor in any short positions.
#                    Price is the price of one share of MSFT.

# Using Adjusted Closing Prices from 2018-01-01 to 2020-09-01

In [3]:
MSFT = pdr.get_data_yahoo("MSFT", start="2017, 12, 1",
                          end= "2020, 9, 1")

# Grabbing data 20 days (20 days the market was open) beforehand so that the 20-day moving average can be calculated for
# the first 20 data entries rather than being filld with "NaN".

MSFT_close = MSFT['Adj Close']
msft_sma20 = MSFT_close.rolling(20).mean()

# Creating a 20-day Simple Moving Average for Microsoft Adjusted-close Prices.
# Will be used to create upper and lower bound for the Bollinger Bands


In [4]:
# This section is to figure out what my bands are going to be.
def get_bands(data):
    
    # These values will be constantly fluctating over time, so I am creating an empty list and for-loops to capture a value at every date.
    st_dev = []
    BB_upper = []
    BB_lower = []
    
    # Standard Deviation of the values used in the 20-Day moving average at the current time.  Uses the values 20-days before all the way up to index i.
    for i in range(20, len(data)):
        st_dev.append(data[i-20:i].std())

    for j in range(len(data[20:])):
        BB_upper.append(data[j] + (st_dev[j] * 2))
        BB_lower.append(data[j] - (st_dev[j] * 2)) 
    # I multpilied by 2 standard deviations so that 95% of the stock data would be contained within the bands.
    # This ensures that the close price when it is outside the band is truly an outlier and very likely to regress back towards the mean.
    
    df = pd.DataFrame()
    df['MSFT'] = MSFT_close[20:]
    df['SMA20'] = msft_sma20[20:]
    df['Upper'] = BB_upper
    df['Lower'] = BB_lower
    
    return df

df = get_bands(msft_sma20)

# Plot of the graph.  I commented it out so that it would not compile at runtime.  I will include a picture of it in the ReadMe.
# 
# plt.figure(figsize=(10.5, 4.5))
# plt.title('MSFT Close Prices with Bollinger Bands')
# plt.plot(df['MSFT'][20:], label = 'MSFT Adjusted Close Prices')
# plt.plot(df['Upper'], label = 'Upper Band')
# plt.plot(df['Lower'], label = 'Lower Band')
# plt.xlabel('Date')
# plt.ylabel('Stock Price (USD)')
# plt.legend(loc = 'upper left')
# plt.show()
# 




In [25]:
# This section is the trading strategy itself.  
# I plan to buy a long position when the stock price dips below the lower bound and sell when it reaches above the upper bound.
    
df = get_bands(msft_sma20)
dataframe = df.drop(columns=['SMA20', 'Upper', 'Lower'])
 
    #Create new empty lists so that I can store the values when I am making transactions
Price = []
Buy_Sell = []
Stock_when_owned = []
log = pd.DataFrame()

bought = 0    #This will be used as a marker to see if I have allocated my money into the stock or not.
    
for i in range(len(MSFT_close)-20):
    # The only two significant possibilities that will influence my Trading Strategy are as follows:
        
    # First Instance (if-statement 1): MSFT close price dips below the lower Bollinger Band
    if df['MSFT'][i] < df['Lower'][i]:
        if bought == 0:    #Checking if it isn't currently owned
            Price.append(dataframe['MSFT'][i])
            Buy_Sell.append("BUY")
            bought = 1
        #Second Instance (if-statement 2): MSFT close price rises above the upper Bollinger Bond 
    if df['MSFT'][i] > df['Upper'][i]:
        if bought == 1:    #Checks to see that we currently own the stock
            Price.append(dataframe['MSFT'][i])
            Buy_Sell.append("SELL")          
            bought = 0
    
    # Final Calculations/Organizational methods in order to display the final table as my output of the function.
    
log['Price'] = Price
log['Buy/Sell'] = Buy_Sell


returns = 0
for i in range(len(log)):
    if Buy_Sell[i] == "SELL":
        returns += (Price[i] - Price[i-1])
print(log)
print()
print("Total Returns in Dollars/Share: ")
print(returns)


         Price Buy/Sell
0    84.174950      BUY
1    90.547447     SELL
2    85.468750      BUY
3    92.758484     SELL
4   103.341148      BUY
5   108.636749     SELL
6   102.477531      BUY
7   106.163269     SELL
8   100.590660      BUY
9   103.376968     SELL
10  118.097771      BUY
11  125.961761     SELL
12  130.287949      BUY
13  136.870834     SELL
14  132.471863      BUY
15  136.852020     SELL
16  131.888504      BUY
17  138.473541     SELL
18  134.548248      BUY
19  136.960770     SELL
20  133.134323      BUY
21  136.565262     SELL
22  134.142853      BUY
23  136.683914     SELL
24  134.834961      BUY
25  138.364792     SELL
26  149.839462      BUY
27  170.989304     SELL

Total Returns in Dollars/Share: 
83.90614318847656
